In [ ]:
import threading
import time
import random
from datetime import datetime
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets

print("✅ All libraries imported successfully!")
print("=" * 50)

✅ All libraries imported successfully!


In [ ]:
class ReadersWritersProblem:
    def __init__(self):
        self.reader_count = 0
        self.waiting_writers = 0
        self.resource_lock = threading.Lock()  # Controls access to shared resource
        self.mutex = threading.Lock()  # Protects reader_count and waiting_writers
        self.can_read = threading.Condition(self.mutex)  # Condition for readers to wait if writers are priority
        self.can_write = threading.Condition(self.mutex)  # Condition for writers (not strictly needed but for clarity)
        self.shared_data = 0
        self.logs = []
        self.stats = {'total_reads': 0, 'total_writes': 0, 'max_concurrent_readers': 0}

    def log_event(self, event):
        timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        log_entry = f"[{timestamp}] {event}"
        self.logs.append(log_entry)
        print(log_entry)

    def reader(self, reader_id, end_time):
        while time.time() < end_time:
            with self.mutex:
                while self.waiting_writers > 0:  # Wait if writers have priority
                    self.log_event(f"🟡 Reader {reader_id} WAITING (Writer priority)")
                    self.can_read.wait()
                self.reader_count += 1
                self.stats['max_concurrent_readers'] = max(self.stats['max_concurrent_readers'], self.reader_count)
            self.log_event(f"📖 Reader {reader_id} READING | Data = {self.shared_data} | Readers = {self.reader_count}")
            self.stats['total_reads'] += 1
            time.sleep(random.uniform(1, 2))
            with self.mutex:
                self.reader_count -= 1
                self.log_event(f"✅ Reader {reader_id} finished | Readers = {self.reader_count}")
                if self.reader_count == 0:
                    self.can_write.notify()  # Notify waiting writers if no readers left
            time.sleep(random.uniform(2, 4))

    def writer(self, writer_id, end_time):
        while time.time() < end_time:
            with self.mutex:
                self.waiting_writers += 1
                self.log_event(f"⏳ Writer {writer_id} WAITING | Waiting Writers = {self.waiting_writers}")
                while self.reader_count > 0:  # Wait for all readers to finish
                    self.can_write.wait()
                self.waiting_writers -= 1
            with self.resource_lock:
                old_value = self.shared_data
                self.log_event(f"✍️ Writer {writer_id} WRITING | Old Data = {old_value}")
                self.shared_data += 1
                time.sleep(random.uniform(1.5, 2.5))
                self.log_event(f"💾 Writer {writer_id} finished | New Data = {self.shared_data}")
                self.stats['total_writes'] += 1
            with self.mutex:
                self.can_read.notify_all()  # Notify waiting readers after write
            time.sleep(random.uniform(3, 5))

print("✅ ReadersWritersProblem class defined successfully!")

✅ ReadersWritersProblem class defined successfully!


In [ ]:
def run_simulation(num_readers=3, num_writers=2, duration=15):
    print("=" * 70)
    print("🚀 STARTING SIMULATION WITH WRITER PRIORITY")
    print("=" * 70)
    print(f"📊 Config: Readers={num_readers}, Writers={num_writers}, Duration={duration}s")
    print("=" * 70)
    rw = ReadersWritersProblem()
    threads = []
    end_time = time.time() + duration
    for i in range(num_readers):
        t = threading.Thread(target=rw.reader, args=(i+1, end_time))
        threads.append(t)
        t.start()
    for i in range(num_writers):
        t = threading.Thread(target=rw.writer, args=(i+1, end_time))
        threads.append(t)
        t.start()
    time.sleep(duration)
    print("\n" + "=" * 70)
    print("📈 STATS")
    print("=" * 70)
    print(f"Total Reads: {rw.stats['total_reads']}")
    print(f"Total Writes: {rw.stats['total_writes']}")
    print(f"Max Concurrent Readers: {rw.stats['max_concurrent_readers']}")
    print(f"Final Data: {rw.shared_data}")
    print("=" * 70)
    return rw

In [ ]:
# Run the simulation
result = run_simulation()

🚀 STARTING SIMULATION WITH WRITER PRIORITY
📊 Config: Readers=3, Writers=2, Duration=15s
[12:27:02.788] 📖 Reader 1 READING | Data = 0 | Readers = 1
[12:27:02.790] 📖 Reader 2 READING | Data = 0 | Readers = 2
[12:27:02.791] 📖 Reader 3 READING | Data = 0 | Readers = 3
[12:27:02.791] ⏳ Writer 1 WAITING | Waiting Writers = 1
[12:27:02.792] ⏳ Writer 2 WAITING | Waiting Writers = 2
[12:27:03.893] ✅ Reader 1 finished | Readers = 2
[12:27:04.338] ✅ Reader 3 finished | Readers = 1
[12:27:04.460] ✅ Reader 2 finished | Readers = 0
[12:27:04.460] ✍️ Writer 1 WRITING | Old Data = 0
[12:27:06.362] 💾 Writer 1 finished | New Data = 1
[12:27:06.865] 🟡 Reader 3 WAITING (Writer priority)
[12:27:07.294] 🟡 Reader 2 WAITING (Writer priority)
[12:27:07.381] 🟡 Reader 1 WAITING (Writer priority)
[12:27:10.703] ⏳ Writer 1 WAITING | Waiting Writers = 2
[12:27:10.703] ✍️ Writer 1 WRITING | Old Data = 1
[12:27:12.723] 💾 Writer 1 finished | New Data = 2
[12:27:12.725] 🟡 Reader 1 WAITING (Writer priority)
[12:27:12.72

[12:27:18.807] 💾 Writer 1 finished | New Data = 3
[12:27:18.808] 🟡 Reader 1 WAITING (Writer priority)
[12:27:18.808] 🟡 Reader 2 WAITING (Writer priority)
[12:27:18.808] 🟡 Reader 3 WAITING (Writer priority)


In [ ]:
import ipywidgets as widgets

def interactive_simulation(readers, writers, duration):
    clear_output(wait=True)
    run_simulation(readers, writers, duration)

widgets.interact(
    interactive_simulation,
    readers=widgets.IntSlider(min=1, max=10, value=3, description='Readers:'),
    writers=widgets.IntSlider(min=1, max=5, value=2, description='Writers:'),
    duration=widgets.IntSlider(min=5, max=30, step=5, value=15, description='Duration (s):')
)

interactive(children=(IntSlider(value=3, description='Readers:', max=10, min=1), IntSlider(value=2, descriptio…

<function __main__.interactive_simulation(readers, writers, duration)>

[12:27:37.950] 💾 Writer 1 finished | New Data = 3
[12:27:37.951] 🟡 Reader 2 WAITING (Writer priority)
[12:27:37.951] 🟡 Reader 1 WAITING (Writer priority)
[12:27:37.952] 🟡 Reader 3 WAITING (Writer priority)


In [ ]:
import pandas as pd

# example logs (agar tumhare paas already list hai to usay use karo)
logs = [
    "Simulation Started",
    "Reader 1 reading",
    "Writer 1 writing",
    "Writer finished",
    "Reader finished"
]

shared_values = [0, 0, 1, 1, 2]

df = pd.DataFrame({
    "step": range(len(logs)),
    "log": logs,
    "shared_data": shared_values
})

df.to_csv("simulation_data.csv", index=False)

print("✅ Data saved to simulation_data.csv")


✅ Data saved to simulation_data.csv
